In [ ]:
# BEFORE RUNNING UPLOAD ALL DATA IN nptel_face_recognition  FOLDER ON DRIVE

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 1.33 ms


In [ ]:
try:
    import face_recognition
except:
    !pip install face_recognition
    import face_recognition

time: 1.44 s


In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
time: 1.43 s


In [ ]:
try:
    import pytesseract
except:
    !pip install pytesseract
    import pytesseract

time: 6.63 ms


In [ ]:
import numpy as np
import time
import pandas as pd
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from PIL import Image, ImageStat
import torch
import time
tt = time.time
st = tt()

time: 454 ms


In [ ]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.1 ms


In [ ]:
# for colab
colab = True
base_dir = "/content/drive/My Drive/nptel_face_recognition"
print(f"os.getcwd(): {os.getcwd()}")
if not os.getcwd() == base_dir:
    os.chdir(base_dir)
    print(f"os.getcwd(): {os.getcwd()}")

os.getcwd(): /content
os.getcwd(): /content/drive/My Drive/nptel_face_recognition
time: 3.34 ms


In [ ]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def make_rgb_bgr(img):
    b,g,r = cv2.split(img)          
    rgb_img = cv2.merge([r,g,b]) 
    return rgb_img

def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 50, hthresh = 200):
    bright = brightness(img)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def get_prof_name(text, title=False):
    hon_list = ['prof','dr']
    ext_list = ['.', ':', ' ', '']
    hon_list = flatten_list([[w+ext for ext in ext_list] for w in hon_list])
    
    word_before_list = ['assosiate', 'adjunct', 'professor', 'faculty', 'dept.', 'department']
    ishon=False
    text = text.split(' ')
    text_copy = text.copy()
    for hon in hon_list:
        if hon in text:       
#             print(f" honorific in text: {text_copy}")
            idx = text.index(hon)
            text = text[idx:]
            if title:
                text = " ".join(text)
                return text
                
            else:
                # get all words before a specified word
                for wb in word_before_list:
                    if wb in text:
                        wbidx = text.index(wb)
                        text = " ".join(text[:wbidx])
                        text = ''.join([x for x in text if x.isalpha() or x in ext_list])  # get rid of non apha chars
#                         print(f" honorific and word_brfore in text: {text_copy}")
                        return text
                
                # if last name is less than 4 letters dont include
                if len(text)>3:
                    if len(text[3])<4:
                        text = text[:3]
                    else:
                        text = text[:4]
                return " ".join(text)
            
    return None
    
def get_name_fom_image(img, tesseract_file_path):
    
    # pytesseract.pytesseract.tesseract_cmd = tesseract_file_path
#     img = subimg(img, u=0.7)

    # Convert the image to gray scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # Performing OTSU threshold 
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) 

    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18)) 

    # Appplying dilation on the threshold image 
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 
    # dilation = thresh1

    # Finding contours 
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,  
                                                     cv2.CHAIN_APPROX_NONE) 

    # Creating a copy of image 
    im2 = img.copy() 
    text = None
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 

        # Cropping the text block for giving input to OCR 
        cropped = im2[y:y + h, x:x + w] 

        # Apply OCR on the cropped image 
        text_sub = pytesseract.image_to_string(cropped)
        text_sub = text_sub.lower().replace("\n"," ").strip()
        if text_sub=='':
            continue
        text_sub_name = get_prof_name(text_sub)
        if text_sub_name is not None:
            text = text_sub_name
            break
    
    return text

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

print(f"torch.cuda.is_available(): {torch.cuda.is_available()}")
if torch.cuda.is_available():
    fr_fl_dict = {"number_of_times_to_upsample":3, "model":"cnn"}
    fr_fe_dict = {"num_jitters":3, "model":"large"}
else:
    fr_fl_dict = {"number_of_times_to_upsample":1, "model":"hog"}
    fr_fe_dict = {"num_jitters":1, "model":"small"}

torch.cuda.is_available(): True
time: 176 ms


In [ ]:
# %%capture cap --no-stderr

# save faces and names
ver = "_with_diversity"
tesseract_file_path = os.path.join(base_dir, "tesseract.exe")
data_dir = os.path.join(base_dir, "data")
known_faces_dir = os.path.join(data_dir, "known_faces"+ver)
full_vedios_dir = os.path.join(data_dir, "full_vedio")
tracebacks_dir = os.path.join(data_dir, "tracebacks")
csv_files_dir = os.path.join(data_dir, "csv_files")

os.makedirs(known_faces_dir, exist_ok = True)
os.makedirs(tracebacks_dir, exist_ok = True)
os.makedirs(csv_files_dir, exist_ok = True)

pkl_files_dir = os.path.join(data_dir, "pkl_files")
nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
folder_name_2_prof_name_dict = {}
folder_name_cnt = 0

filter_playlist_list = ["Instability and Transition of Fluid Flows"]
save_img_threshold = 0.25

num_channels = 5
num_playlists = 5
num_vedios = 5

ori_num_faces = 5
ori_num_vedios_for_faces = 2

tfacef=0
tnamef=0
tnamenf=0
results_dict_list=[]
playlist_2_prof_name_dict={}
# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[:num_channels]:
    print(f'\n\n\nfor nptel_channel_name {nptel_channel_name}')
    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    playlist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]

    print(f'number of playlist: {len(playlist_path_list)}')
    vedio_ext = '.mp4'
    
    facef=0
    namef=0
    namenf=0
    facef_playlist_names_list=[]
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    for playlist_path in playlist_path_list[:num_playlists]:
        gotname = False
        gotface = False
        gotallfaces =False
        playlistn = playlist_path.split('/')[-1]
        print(f"\nplaylist: {playlistn}")
        if playlistn in filter_playlist_list:
            continue
            
        vedio_path_list = get_extension_files(playlist_path, ext=vedio_ext)
        print(f"number of vedios: {len(vedio_path_list)}")
        
        vst = 0*60
        search_interval = 1*60
        vet = search_interval

        num_f_per_sec = 1
        num_f_inc = 1
        
        num_faces_found = 0
        num_vedios_for_faces = ori_num_vedios_for_faces
        prof_folder_made = False

        text_sub = get_prof_name(playlistn.lower(), title=True)
        if text_sub is not None:
            print(f'name: {text_sub} in the title')
            prof_name = text_sub 
            gotname = True

        for vedio_path in vedio_path_list[:num_vedios]:
            if gotallfaces:
                continue
            else:
                if gotname:
                    vst+=search_interval
                    vet+=search_interval
                    num_f_per_sec = 1
#                     num_f_per_sec+=num_f_inc
                    pass
                else:
                    num_f_per_sec+=num_f_inc
            
            num_faces = ori_num_faces
            vname = vedio_path.split('/')[-1]
            print(f"vedio: {vname}")
            print(f"(vst, vet, num_f_per_sec): {(vst, vet, num_f_per_sec)}")
            video_capture = cv2.VideoCapture(vedio_path)
            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                ret, frame = video_capture.read()

                # convert to rgb
                rgb_frame = frame[:, :, ::-1]

                if i%int(fps/num_f_per_sec)==0:
    #                 print(f"time(sec): {int(i/fps)}")
                    if not gotname:
                        text = get_name_fom_image(frame, tesseract_file_path)
                        if text is not None:
                            print(f'name: {text} at time: {int(i/fps)}')
                            prof_name = text
                            gotname = True

                    else:
                        if not is_valid_img(frame):
                            continue
                            
                        face_locations = face_recognition.face_locations(rgb_frame, **fr_fl_dict)
                        if len(face_locations)==1:
                            
                            # check if encoding is empty
                            face_encoding = face_recognition.face_encodings(rgb_frame, known_face_locations = face_locations, **fr_fe_dict)
                            if not face_encoding:
                                continue
                            
                            face_encoding = face_encoding[0]
                            
                            if not prof_folder_made:
                                folder_name_2_prof_name_dict[folder_name_cnt] = prof_name

                                # some chaectors in name make invalid dir names
                                prof_faces_dir = os.path.join(known_faces_dir, str(folder_name_cnt))
                                folder_name_cnt+=1
                                os.makedirs(prof_faces_dir, exist_ok = True)
                                prof_folder_made = True
                            
                            # save image farther from current saved image using face_recognition
                            known_face_encodings = []
                            known_encoding_path_list = get_extension_files(prof_faces_dir, ext='.pkl') 
                            if known_encoding_path_list!=[]:
                                for known_encoding_path in known_encoding_path_list:
                                    enc = load_from_pickle(encoding_path)
                                    known_face_encodings.append(enc)
                                    
                                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                                if min(face_distances) < save_img_threshold:
                                    continue
                                
                            image_path = os.path.join(prof_faces_dir, str(num_vedios_for_faces*10+num_faces)+'.jpg')
                            encoding_path =  os.path.join(prof_faces_dir, str(num_vedios_for_faces*10+num_faces)+'.pkl')
                            cv2.imwrite(image_path, frame)
                            save_as_pickle(encoding_path, face_encoding)
                            gotface=True
                            num_faces_found+=1
                            num_faces-=1
#                             print(f'face at time: {int(i/fps)}')
                            if num_faces<=0:
                                num_vedios_for_faces-=1
                                if num_vedios_for_faces==0:
                                    gotallfaces = True
                                    break

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            video_capture.release()
            cv2.destroyAllWindows()

            results_dict = {"vname": vname,
                            "nptel_channel_name": nptel_channel_name, 
                            "playlistn": playlistn,
                            "vedio_path": vedio_path,
                            "time": tt()-st,
                            "gotallfaces": gotallfaces,
                            "gotname": gotname,
                            "gotface": gotface,
                            }
            results_dict_list.append(results_dict)

        if gotallfaces:
            facef+=1
            print(f"all faces found for {playlist_path}")
            facef_playlist_names_list.append(playlistn)
            playlist_2_prof_name_dict[playlistn] = prof_name
            
        else:
            if gotname:
                print(f"num of faces found: {num_faces_found} for {playlist_path}")
                namef+=1
                namef_playlist_names_list.append(playlistn)
                playlist_2_prof_name_dict[playlistn] = prof_name
            else:
                print(f"face and name not found for {playlist_path}")
                playlist_2_prof_name_dict[playlistn] = None
                namenf_playlist_names_list.append(playlistn)
                namenf+=1
    
    tfacef+=facef
    tnamef+=namef
    tnamenf+=namenf
    print(f"\nfor channel: {nptel_channel_name}  faces found: {facef}, names found: {namef}, names not found: {namenf}")    

    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)

print(f"\ntotal faces found: {tfacef}, names found: {tnamef}, names not found: {tnamenf}")
results_dict_csv = pd.DataFrame(results_dict_list)
results_dict_csv.to_csv(os.path.join(csv_files_dir, "get_names_faces_results"+ver+".csv"))






for nptel_channel_name Aerospace Engineering
number of playlist: 5

playlist: Jet and Rocket Propulsion
number of vedios: 5
vedio: bcBrCQGI1tY.mp4
(vst, vet, num_f_per_sec): (0, 60, 2)
name: dr. a. kushari at time: 12
vedio: UcDNM18TkiU.mp4
(vst, vet, num_f_per_sec): (60, 120, 1)
all faces found for /content/drive/My Drive/nptel_face_recognition/data/full_vedio/Aerospace Engineering/Jet and Rocket Propulsion

playlist: Instability and Transition of Fluid Flows

playlist: Turbomachinery Aerodynamics
number of vedios: 5
vedio: VCWzszufr7I.mp4
(vst, vet, num_f_per_sec): (0, 60, 2)
name: prof. ee et at time: 11
vedio: U6ipqqt6Spc.mp4
(vst, vet, num_f_per_sec): (60, 120, 1)
all faces found for /content/drive/My Drive/nptel_face_recognition/data/full_vedio/Aerospace Engineering/Turbomachinery Aerodynamics

playlist: Introduction to Aerodynamics
number of vedios: 4
vedio: FKH4DjMdk9I.mp4
(vst, vet, num_f_per_sec): (0, 60, 2)
name: prof. k. p. sinhamahapatra at time: 12
vedio: VhybIE82B6M.m

NameError: ignored

time: 51min 18s


In [ ]:
prof_name_2_playlist_dict = {v: k for k, v in playlist_2_prof_name_dict.items()}
save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_playlist_dict"+ver+".pkl"), prof_name_2_playlist_dict)
save_as_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict"+ver+".pkl"), playlist_2_prof_name_dict)


prof_name_2_folder_name_dict = {v: k for k, v in folder_name_2_prof_name_dict.items()}
save_as_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict"+ver+".pkl"), folder_name_2_prof_name_dict)
save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_folder_name_dict_diversity"+ver+".pkl"), prof_name_2_folder_name_dict)

time: 24.1 ms
